In [1]:
!pip install dnspython
from pandas import DataFrame
from pymongo import MongoClient
import os

#Broadcaster data
client = MongoClient(os.environ.get('MONGO_BROADCASTERDATA_URI'))
db = client["hybrid-id"]

    
print("List of collections \n--------------------")

#list collection names
for x in db.list_collection_names():
    print(x)

List of collections 
--------------------
deviceusers
devices


In [2]:
deviceusers = db.deviceusers.find( {} ) #get all documents from deviceusers collection
list_cur = list(deviceusers) #cursor to list
df_deviceusers = DataFrame(list_cur) #cursor to dataframe

In [3]:
df_deviceusers.drop(columns=['__v', 'chlist', 'mid'], inplace=True) 

In [4]:
df_deviceusers.head()

,_id,uaIpHash
0,5e3d270effc977001ea26de7,cddcc768d93f853302995e33d2a929564ba269c1
1,5e3d2712ffc977001ea26df5,e7c4a5c04071291ab09c626b7f512e51d7668e9e
2,5e3d2712ffc977001ea26df3,ad7c5255c478cc541e29834429668b9836063d87
3,5e3d271affc977001ea26e0b,4bb42b4e3dcdaff864ce431db1b2579af3c29ba5
4,5e3d271dffc977001ea26e19,b488d091653eb57b62a91ad0f74207ea9cadd2fa


In [5]:
df_deviceusers.columns = ['DeviceUser_id', 'UaIpHash']
df_deviceusers.head()

,DeviceUser_id,UaIpHash
0,5e3d270effc977001ea26de7,cddcc768d93f853302995e33d2a929564ba269c1
1,5e3d2712ffc977001ea26df5,e7c4a5c04071291ab09c626b7f512e51d7668e9e
2,5e3d2712ffc977001ea26df3,ad7c5255c478cc541e29834429668b9836063d87
3,5e3d271affc977001ea26e0b,4bb42b4e3dcdaff864ce431db1b2579af3c29ba5
4,5e3d271dffc977001ea26e19,b488d091653eb57b62a91ad0f74207ea9cadd2fa


In [15]:
df = df_deviceusers.set_index('DeviceUser_id')
df.head()

,UaIpHash
DeviceUser_id,
5e3d270effc977001ea26de7,cddcc768d93f853302995e33d2a929564ba269c1
5e3d2712ffc977001ea26df5,e7c4a5c04071291ab09c626b7f512e51d7668e9e
5e3d2712ffc977001ea26df3,ad7c5255c478cc541e29834429668b9836063d87
5e3d271affc977001ea26e0b,4bb42b4e3dcdaff864ce431db1b2579af3c29ba5
5e3d271dffc977001ea26e19,b488d091653eb57b62a91ad0f74207ea9cadd2fa


In [17]:
df.to_csv('deviceuser2.csv')

In [6]:
!pip install snowflake-connector-python
import pandas as pd
import snowflake.connector

     |████████████████████████████████| 12.2MB 6.6MB/s 
     |████████████████████████████████| 112kB 38.6MB/s 
     |████████████████████████████████| 194kB 44.4MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 13.7MB 278kB/s 
     |████████████████████████████████| 337kB 50.5MB/s 
     |████████████████████████████████| 133kB 53.6MB/s 
     |████████████████████████████████| 2.6MB 38.1MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 133kB 54.6MB/s 
     |████████████████████████████████| 7.2MB 38.3MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
ERROR: botocore 1.19.53 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [8]:
conn = snowflake.connector.connect(
                user="os.environ.get('SNOWFLAKE_USER')",
                password="os.environ.get('SNOWFLAKE_PASS')",
                account="os.environ.get('SNOWFLAKE_ACC')",
                database="os.environ.get('SNOWFLAKE_DB')",
                schema="os.environ.get('SNOWFLAKE_SCHEMA')")

In [9]:
cur = conn.cursor()

In [10]:
cur.execute('USE WAREHOUSE AFTERSPOT_WH')

In [18]:
cur.execute("put file://{} @{} auto_compress=true".format('deviceuser2.csv', 'TEST'))

In [21]:
cur.execute("COPY INTO DIM_DEVICEUSER FROM @test/deviceuser2.csv.gz FILE_FORMAT=(TYPE=csv field_delimiter=',' skip_header=1) ON_ERROR = 'CONTINUE'")